This notebook applies to drx+tet structures such as the partially disordered spinel where Li+, Mn2+, and Vacancies occupy octahedral and tetrahedral sublattices. On the octahedral sublattice there are Li+, Mn2+, Mn3+, Mn4+, and Vacancies. The anion sublattice contains only oxygen. 

Note that canonical MC with overlapping sublattices was written primarily with Mn disordering in mind. It has been tested for other systems (e.g. Z. Jadidi et al., in preparation, (2022)) but not widely. As a result, please use with care. 

Authors: Julia Yang, Tina Chen

Date: May 2022

## 1. Initialize necessary data (starting structure and cluster expansion model)

In [1]:
import json
import numpy as np
from smol.io import load_work
from smol.cofe.expansion import ClusterExpansion 

starting_spinel_file = 'data/limn2o4_spinel.json'
mson = 'data/lmof_drx_tet.mson'

# load the starting configuration which is a spinel structure
from pymatgen.core.structure import *
with open(starting_spinel_file, 'r') as f: 
    init_struct = Structure.from_dict(json.load(f))

# this is the supercell shape of the spinel structure
m = np.matrix([[2,2,-2],[-2,2,2],[2,-2,2]]) 

# run a small cell for now 
transformation = np.identity(3)
supercell_matrix = m @ transformation

# load CE
work = load_work(mson)
expansion = work['ClusterExpansion']

## 2. Create the Ensemble and use in MC sampling 

If you want to save the occupancies for later analysis of 16c or 16d sites, you *must* save current_occupancy now, as generating this processor via `ensemble.processor` does not maintain bit order. In other words, the 16c and 16d sites index to different indices in `occupancy` if you analyze the 16c or 16d sites from generating another new processor later, say processor2 for clarity, via `processor2.occupancy_from_structure(init_struct)`. This is not necessarily a bug. Just be aware that one must exercise caution when comparing bit-by-bit. 

`Mn_swap_probability` is the probability of choosing a flip from `Mn_flip_table` which contain disproportionation reactions possible in Mn-oxides. See John Reed's thesis for more information.

In [2]:
Mn_flip_table = {('Mn2+', 'Mn2+'): ['None'],
                 ('Mn2+', 'Mn3+'): ['swap'],
                 ('Mn3+', 'Mn2+'): ['swap'],
                 ('Mn2+', 'Mn4+'): ['dispropA', 'swap'],
                 ('Mn4+', 'Mn2+'): ['dispropB', 'swap'],
                 ('Mn3+', 'Mn3+'): ['dispropC', 'dispropD'],
                 ('Mn3+', 'Mn4+'): ['swap'],
                 ('Mn4+', 'Mn3+'): ['swap'],
                 ('Mn4+', 'Mn4+'): ['None']}

In [3]:
from smol.moca import Sampler 
from smol.moca.ensemble.canonical import CanonicalEnsemble

ALLOW_CROSSOVER=True
STEP_TYPE='tableswapper'
NUMBER_OF_PROPOSALS=20 # quick T scan parameter 
START_T = 300 
END_T = 400 # quick T scan parameter 
STEP_T = 100

ensemble = CanonicalEnsemble.from_cluster_expansion(expansion, 
                                                    supercell_matrix)

current_occupancy = ensemble.processor.occupancy_from_structure(init_struct)

for T in range(START_T, END_T+1, STEP_T): 
    sampler = Sampler.from_ensemble(kernel_type='Metropolis',
                                    ensemble=ensemble,
                                    temperature=T,
                                    step_type=STEP_TYPE,
                                    allow_crossover=True,
                                    Mn_swap_probability=0.5,
                                    swap_table=None, 
                                    Mn_flip_table=Mn_flip_table,
                                    run_initial_step=False,
                                )
    sampler.run(NUMBER_OF_PROPOSALS,
               initial_occupancies=current_occupancy)



## 4. Save the samples for future analysis

In [4]:
save_dict = {'features': sampler.samples.get_feature_vectors().tolist(),
             'occupancies': sampler.samples.get_occupancies().tolist(),
             'efficiency': sampler.samples.sampling_efficiency}
#with open('canonical_mc_overlapping_sublattices_example.json', 'w') as f: 
#    json.dump(save_dict, f)